# Designing Analysis Classes

## Points to demonstrate

  * Data specification
      * Filesets and Fields
      * Frequency
      * valid formats
      * pipeline getter
  * Parameter specification
      * Switches
      * Parameters
  * Pipeline getters
      * name_maps
      * requirements
      * citations
      * inputs
          * from spec
          * from prev node
      * output to spec
      * Data formats and implicit conversion      